# Clean dataset

In [1]:
import pandas as pd
import numpy as np

pd.options.mode.chained_assignment = None

/Users/ilenapeng/.pyenv/versions/3.9.7/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
df = pd.read_csv('../data/processed/pipeline_incidents_2010_present_all.csv')

/Users/ilenapeng/.pyenv/versions/3.9.7/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1,15,16,18,19,20,21,22,25,26,27,28,29,30,31,32,33,36,56,57,58,63,64,65,72,73,75,76,77,78,84,85,87,89,92,93,94,95,101,102,103,104,105,106,108,109,116,120,148,149,151,152,154,171,172,173,193,194,197,198,205,209,210,214,217,218,219,221,222,223,225,228,230,231,233,234,238,239,240,250,251,252,256,257,261,262,263,265,266,268,269,276,277,278,279,280,281,286,287,290,293,294,295,296,297,300,301,303,304,305,306,307,310,311,313,314,315,316,317,318,319,320,321,322,324,325,326,327,329,330,331,332,333,334,335,336,337,340,341,342,343,344,345,347,348,351,354,355,356,357,359,360,361,362,363,365,366,367,368,369,371,372,373,374,375,376,377,378,379,380,384,386,387,388,389,391,393,394,395,399,400,401,402,407,411,421,422,423,427,428,429,430,432,433,434,435,436,439,440,441,442,443,446,447,449,451,452,454,455,456,457,460,462,463,464,465,466,472,473,474,480,482,483,484,485,486,487,48

## Keep only relevant columns

In [3]:
columns_keep = [
    'datafile_as_of',
    'ff',
    'significant',
    'serious',
    'report_number',
    'supplemental_number',
    'report_received_date',
    'report_type',
    'operator_id',
    'name',
    'operator_street_address',
    'operator_city_name',
    'operator_state_abbreviation',
    'operator_postal_code',
    'local_datetime',
    'time_zone',
    'daylight_savings_ind',
    'iyear',
    'location_street_address',
    'location_city_name',
    'location_county_name',
    'location_state_abbreviation',
    'location_postal_code',
    'location_latitude',
    'location_longitude',
    'total_cost',
    'total_cost_current',
    'injury_ind',
    'injure',
    'num_pub_evacuated',
    'fatal',
    'cause',
    'cause_details',
    'material_involved',
    'material_details',
    'narrative',
    'unintentional_release',
    'installation_year'
]

In [4]:
df_final = df[columns_keep]
df_final.columns

Index(['datafile_as_of', 'ff', 'significant', 'serious', 'report_number',
       'supplemental_number', 'report_received_date', 'report_type',
       'operator_id', 'name', 'operator_street_address', 'operator_city_name',
       'operator_state_abbreviation', 'operator_postal_code', 'local_datetime',
       'time_zone', 'daylight_savings_ind', 'iyear', 'location_street_address',
       'location_city_name', 'location_county_name',
       'location_state_abbreviation', 'location_postal_code',
       'location_latitude', 'location_longitude', 'total_cost',
       'total_cost_current', 'injury_ind', 'injure', 'num_pub_evacuated',
       'fatal', 'cause', 'cause_details', 'material_involved',
       'material_details', 'narrative', 'unintentional_release',
       'installation_year'],
      dtype='object')

## Create variable with decades

In [5]:
# create decades variable
conditions = [
    (df['installation_year'] >= '1900-01-01') & (df['installation_year'] < '1910-01-01'),
    (df['installation_year'] >= '1910-01-01') & (df['installation_year'] < '1920-01-01'),
    (df['installation_year'] >= '1920-01-01') & (df['installation_year'] < '1930-01-01'),
    (df['installation_year'] >= '1930-01-01') & (df['installation_year'] < '1940-01-01'),
    (df['installation_year'] >= '1940-01-01') & (df['installation_year'] < '1950-01-01'),
    (df['installation_year'] >= '1950-01-01') & (df['installation_year'] < '1960-01-01'),
    (df['installation_year'] >= '1960-01-01') & (df['installation_year'] < '1970-01-01'),
    (df['installation_year'] >= '1970-01-01') & (df['installation_year'] < '1980-01-01'),
    (df['installation_year'] >= '1980-01-01') & (df['installation_year'] < '1990-01-01'),
    (df['installation_year'] >= '1990-01-01') & (df['installation_year'] < '2000-01-01'),
    (df['installation_year'] >= '2000-01-01') & (df['installation_year'] < '2010-01-01'),
    df['installation_year'] >= '2010-01-01'
]

outputs = [
    '1900-1909', '1910-1919', '1920-1929', '1930-1939', '1940-1949', '1950-1959', '1960-1969', '1970-1979',
    '1980-1989', '1990-1999', '2000-2009', '>2010'
]

df_final['decade'] = np.select(conditions, outputs, np.nan)
df_final.decade.unique()

array(['nan', '1990-1999', '1970-1979', '1980-1989', '1960-1969',
       '2000-2009', '1950-1959', '1940-1949', '>2010', '1920-1929',
       '1930-1939', '1900-1909', '1910-1919'], dtype=object)

## Create variable of current pipeline age at time of incident

In [6]:
# convert to datetime
df_final['installation_year'] = pd.to_datetime(df_final['installation_year'], format='%Y-%m-%d')

In [7]:
df_final['year_dt'] = df_final.installation_year.dt.year

In [8]:
# convert to float to allow us to subtract iyear and installation_year
df_final['iyear'] = df_final.iyear.astype('float', errors='ignore')

In [9]:
# create pipeline_age variable
df_final['pipeline_age'] = df_final['iyear'] - df_final['year_dt']

In [10]:
df_final.to_csv('../data/processed/pipeline_incidents_2010_present_all_CLEAN.csv', index=False)